# publishing datasette to vercel

release a datasette interface on vercel for our assistive technology dataset.

the activities of this notebook require [`pixi`](https://pixi.sh) to be installed.

In [1]:
    import os, json, sys; from pathlib import Path

In [2]:
    BIN = (PY := Path(sys.executable)).parent
    VERCEL = (TARGET := Path("vercel-app")) / "vercel.json"

## build the publishable vercel assets

use the datasette command line to generate the vercel assets. this requires https://github.com/simonw/datasette-publish-vercel/.

In [3]:
    !rm -rf $TARGET
    !pixi r python -m datasette publish vercel at.sqlite --project=a11yhood --generate-dir $TARGET && ls $TARGET
    !cat $VERCEL

Your generated application files have been written to:                                   
    /Users/tonyfast/research/vercel-app

To deploy using Vercel, run the following:
    cd /Users/tonyfast/research/vercel-app
    vercel --prod
at.sqlite        index.py         requirements.txt vercel.json
{
    "name": "a11yhood",
    "version": 2,
    "builds": [
        {
            "src": "index.py",
            "use": "@vercel/python@3.0.7"
        }
    ],
    "routes": [
        {
            "src": "(.*)",
            "dest": "index.py"
        }
    ]
}

we get errors when using the default settings like `"use": "@vercel/python@3.0.7"` so we patch it with a newer version "use": "@vercel/python@4.7.2"

In [4]:
    vercel = json.loads(VERCEL.read_text())
    for build in vercel["builds"]:
        if build["use"].startswith("@vercel/python"):
            build["use"] = "@vercel/python@4.7.2"
    VERCEL.write_text(json.dumps(vercel, indent=2))

209

## deploy the vercel app

In [5]:
    if CI := (os.environ.get("CI")):
        !pixi r vercel $TARGET --prod --yes